# Ordering


## Warning: This notebook will place live orders

Use a papertrading account when running this notebook.


In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=13)

<IB connected to 127.0.0.1:7497 clientId=13>

Create a contract and a market order:

In [2]:
contract = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 100)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [3]:
trade = ib.placeOrder(contract, order)

trade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=23, action='BUY', totalQuantity=100), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 50, 146947, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [4]:
ib.sleep(1)
trade.log

[TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 50, 146947, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 50, 275604, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='Fill 100.0@170.56'),
 TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 50, 275898, tzinfo=datetime.timezone.utc), status='Filled', message='')]

``trade`` will also available from ``ib.trades()``:

In [5]:
assert trade in ib.trades()

Likewise for ``order``:

In [6]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [7]:
limitOrder = LimitOrder('BUY', 100, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=24, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 51, 380610, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

``status`` will change from "PendingSubmit" to "Submitted":

In [8]:
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

In [9]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [10]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=24, action='BUY', totalQuantity=100, lmtPrice=0.1), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=611523718, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 51, 380610, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 51, 494123, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 52, 559908, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify')])

And now cancel it:

In [11]:
ib.cancelOrder(limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=24, action='BUY', totalQuantity=100, lmtPrice=0.1), orderStatus=OrderStatus(status='Submitted', remaining=100.0, permId=611523718, clientId=13), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 51, 380610, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 51, 494123, tzinfo=datetime.timezone.utc), status='Submitted', message=''), TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 52, 559908, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify'), TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 52, 618875, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')])

In [12]:
limitTrade.log

[TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 51, 380610, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 51, 494123, tzinfo=datetime.timezone.utc), status='Submitted', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 52, 559908, tzinfo=datetime.timezone.utc), status='Submitted', message='Modify'),
 TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 52, 618875, tzinfo=datetime.timezone.utc), status='PendingCancel', message=''),
 TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 52, 669207, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')]

Some note of warning: reqContractDetails can happily report that a contract is unique,
while placeOrder() will complain that the contract is ambiguous. An example is INTC:

In [13]:
intc = Stock('INTC', 'SMART', 'USD')

assert len(ib.reqContractDetails(intc)) == 1

intcOrder = LimitOrder('BUY', 100, 0.05)
trade = ib.placeOrder(intc, intcOrder)
ib.sleep(1);

ERROR:ib_insync.wrapper:Error 200, reqId 26: The contract description specified for INTC is ambiguous.


In [14]:
assert trade.orderStatus.status == 'Cancelled'

So it is always a good
idea to a qualify a contract before trading it.

In [15]:
ib.qualifyContracts(intc)

intcOrder = LimitOrder('BUY', 100, 0.05)
trade = ib.placeOrder(intc, intcOrder)

In [16]:
ib.cancelOrder(intcOrder)

Trade(contract=Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), order=LimitOrder(orderId=28, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 54, 114338, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2017, 11, 20, 18, 21, 54, 170613, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')])

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [17]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

CPU times: user 5.42 ms, sys: 108 µs, total: 5.53 ms
Wall time: 208 ms


What are our positions?

In [18]:
ib.positions()

[Position(account='DU15143', contract=Contract(conId=265598, symbol='AAPL', secType='STK', exchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), position=600.0, avgCost=170.56833333333333)]

What's the total of commissions paid today?

In [19]:
sum(fill.commissionReport.commission for fill in ib.fills())

5.0

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [20]:
order = MarketOrder('SELL', 200)
ib.whatIfOrder(contract, order)

OrderState(status='PreSubmitted', initMargin='17056.5', maintMargin='17056.5', equityWithLoan='499993.0', commission=1.0, commissionCurrency='USD')

In [21]:
ib.disconnect()